In [1]:
# this ** > /dev/null 2>&1 ** means that redirect the verbose output to empty that's it. Nothing important
!sudo apt install  -qq tesseract-ocr -y > /dev/null 2>&1
!sudo apt install -qq libtesseract-dev -y > /dev/null 2>&1
!pip install pytesseract lancedb openai xformers -qq

# !pip install python-Levenshtein "python-doctr[torch]" tf2onnx mplcursors  -qq


!pip install "fastapi" nest-asyncio pyngrok uvicorn -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.1 MB/s eta 0:00:00


# Must DO:
1. Go and Signup at [`ngrok`](https://dashboard.ngrok.com/signup) (One time thing)
2. After Successful login go to `https://dashboard.ngrok.com/get-started/setup/linux` and you'll see something like the below code line. Run it with you token
3. Just run every line as it is
4. Once your app will run it'll show random url like  `Public URL:https://2127-35-186-148-235.ngrok-free.app` (it'll change everytime you run the code)
5. Paste the above URL in your local `config.json -> transcription -> API_ENDPOINT`. If it is empty there, your local will model in your system
6. Run your local app as usual given in directory

In [ ]:
!ngrok config add-authtoken YOUR_TOKEN_HERE

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# Load Models
**NOTE**: If you load the `Audio` model before the `OCR`, somehow it alsays gives error for I god knows what reasons

1. Whisper exposedat `/transcribe` Endpoint
2. Image OCR expose at `/ocr`

## Image OCR Model

Switching to `tesseract` because `trocr` is heavy and if it's a screenshot (straight, plain clear typed letters), `tesseract` does an amazing job and it runs on CPU

In [3]:
# from doctr.models import ocr_predictor
# import os

# ocr_model = ocr_predictor(pretrained=True,
#                           assume_straight_pages = True,
#                           straighten_pages = False)# .cuda().half()

# async def perform_ocr(image):
#   """
#   Expects PIL image but the moodel needs a [np array of image]
#   """
#   try:
#     list_of_image = [np.array(image)]
#     json_result = await asyncio.to_thread(ocr_model, list_of_image)

#     lines = []
#     for item in json_result.export()["pages"][0]["blocks"][0]["lines"]:
#         words = []
#         for word_item in item["words"]:
#           words.append(word_item["value"])

#         lines.append(" ".join(words))

#     return "\n".join(lines)

#   except Exception as e:
#     logger.error(f"OCR processing error: {e}")
#     return e


import pytesseract

async def perform_ocr(image):
  try:
    ocr_result = await asyncio.to_thread(pytesseract.image_to_string, image)
    return ocr_result

  except Exception as e:
    logger.error(f"OCR processing error: {e}")
    return e


## Audio Model

In [4]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch
from fastapi import HTTPException
import numpy as np

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

# Whisper settings
WHISPER_LANGUAGE = "english"
TRANSCRIPTION_MODEL_NAME = "openai/whisper-large-v3-turbo"
MAX_SENTENCE_CHARACTERS = 128

device_name = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")
device = torch.device(device_name)
torch_dtype = torch.bfloat16

# ------ Transcription Helpers ------
transcription_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    TRANSCRIPTION_MODEL_NAME, torch_dtype=torch_dtype, low_cpu_mem_usage=True
)
transcription_model.to(device)

processor = AutoProcessor.from_pretrained(TRANSCRIPTION_MODEL_NAME)

transcription_pipeline = pipeline(
    "automatic-speech-recognition",
    model=transcription_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s = 30, #  min(LENGTH_IN_SEC, 30)
    torch_dtype=torch_dtype,
    device=device,
)


async def process_transcription(audio_array: np.ndarray, sample_rate: int = 16000, language: str = "english", return_timestamps: bool = False):
    """
    Process the audio array through the transcription pipeline.
    """
    try:
        return await asyncio.to_thread(
            transcription_pipeline,
            {"array": audio_array, "sampling_rate": sample_rate},
            return_timestamps=return_timestamps,
            generate_kwargs={
                "language": language,
                "return_timestamps": return_timestamps,
                "max_new_tokens": MAX_SENTENCE_CHARACTERS
            }
        )
    except Exception as e:
        logger.error(f"Transcription processing error: {e}")
        raise HTTPException(status_code=500, detail=f"Transcription processing failed: {str(e)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

## Python Leetcode RAG model
Builds DB using `LanceDB` and search the similar Python Q-A

In [14]:
!git clone https://github.com/Garvit244/Leetcode/

import os
from sentence_transformers import SentenceTransformer
import torch
import lancedb
import asyncio

embed_model = SentenceTransformer("dunzhang/stella_en_400M_v5", trust_remote_code=True).to("cuda" if torch.cuda.is_available() else "cpu").half()


def find_py_files(root_dir = "./Leetcode"):
    py_files = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for file in filenames:
            if file.endswith(".py"):
                py_files.append(os.path.join(dirpath, file))
    return py_files

all_files = find_py_files()

data = []
errors = []
docs = [] # same as "text"

for file in all_files:
  with open(file, "r") as f:  text = f.read().lstrip("'''").lstrip('"""')
  if "'''" in text and text.count("'''") == 1: text, code = text.split("'''")
  elif '"""' in text and text.count('"""') == 1: text, code = text.split('"""')
  else:
    errors.append(file)
    continue

  docs.append(text.strip())
  data.append({"problem_statement": text.strip(), "python_code": code.strip(), "problem_num": file.split("/")[-1].replace(".py", "")})

doc_embeddings = embed_model.encode(docs)

for index, item in enumerate(data):
  item["vector"] = doc_embeddings[index]

db = lancedb.connect("./code_db")
code_table = db.create_table("code_table", data)



async def search_db(query: str, model, table, query_prompt_name="s2p_query", top_k=3, max_dist=0.33):
    query_embeddings = model.encode([query], prompt_name=query_prompt_name)[0]  # input went as list

    # Run the synchronous search in a separate thread
    result = await asyncio.to_thread(
        lambda: table.search(query_embeddings)
        .metric("cosine")
        .limit(top_k)
        .where(f"_distance <= {max_dist}").to_list())

    output = ""
    if len(result):
      output = "Below are some similar coding problem and their python code solution to give you context on how to solve the given problem:\n\n"
      for index, item in enumerate(result):
        output += f"""\n#Reference - {index+1}\n\n##Question:\n{item["problem_statement"]}\n\n##Python Code:\n{item["python_code"]}\n"""

    return output.strip()

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## App Code

In [15]:
import asyncio, re, torch
from fastapi import FastAPI, WebSocket, WebSocketDisconnect, HTTPException
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from typing import List, Annotated
import asyncio
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import logging
from pydantic import BaseModel
import json
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import numpy as np
import base64
from typing import Optional
import asyncio
from transformers import pipeline
from fastapi import UploadFile, File, Body
import io
import numpy as np
from PIL import Image
from typing import Annotated


app = FastAPI()
app.add_middleware(
    CORSMiddleware, allow_origins=['*'], allow_credentials=True, allow_methods=['*'], allow_headers=['*'])

class AudioRequest(BaseModel):
    audio_data: str  # Base64 encoded audio data

class RAGRequest(BaseModel):
    query: str  # Incoming question
    top_k: Optional[int] = 3 # How many similar to fetch
    max_dist : Optional[float] = 0.33 # greater than this cosine distance won't be considered

class RAGResponse(BaseModel):
    text: str
    error: Optional[str] = None

class TranscriptionResponse(BaseModel):
    text: str
    timestamps: Optional[list] = None
    error: Optional[str] = None

class ImageData(BaseModel):
    image_bytes: str

class OCRResponse(BaseModel):
    text: str
    error: Optional[str] = None


@app.post("/transcribe", response_model=TranscriptionResponse)
async def transcribe_audio(request: AudioRequest):
    """
    Endpoint to receive audio data and return transcription.

    Expects base64 encoded audio data in the request body.
    Returns transcription text and optional timestamps.
    """
    try:
        # Decode base64 audio data
        try:
            audio_bytes = base64.b64decode(request.audio_data)
        except Exception as e:
            raise HTTPException(status_code=400, detail="Invalid base64 audio data")

        # Convert to numpy array (assuming 16-bit PCM audio)
        try:
            audio_array = np.frombuffer(audio_bytes, np.int16).astype(np.float32) / 32768.0
        except Exception as e:
            raise HTTPException(status_code=400, detail="Failed to process audio data")

        # Validate audio data
        if len(audio_array) == 0:
            raise HTTPException(status_code=400, detail="Empty audio data")

        # Process transcription
        result = await process_transcription(audio_array)

        return TranscriptionResponse(text=result["text"])

    except HTTPException as he:
        raise he
    except Exception as e:
        logger.error(f"Unexpected error in transcribe_audio: {e}")
        raise HTTPException(status_code=500, detail=f"Internal server error: {str(e)}")


@app.post("/OCR", response_model=OCRResponse)
async def ocr_image(image_object: ImageData):
    """
    Endpoint to perform OCR on an uploaded image.
    """
    try:
        try:
            image = Image.open(io.BytesIO(base64.b64decode(image_object.image_bytes)))
        except Exception as e:
            raise HTTPException(status_code=400, detail=f"Error processing image data: {str(e)}")

        ocr_result = await perform_ocr(image) # model expects a list of numpy [H, W, C] uint8 images

        return OCRResponse(text= ocr_result)

    except HTTPException as he:
        raise he
    except Exception as e:
        logger.error(f"Unexpected error in ocr_image: {e}")
        raise HTTPException(status_code=500, detail=f"Internal server error: {e}")


@app.post("/python-leetcode-RAG", response_model = RAGResponse)
async def search_code(request: RAGRequest):
    try:
        result = await search_db(
            request.query,
            embed_model,
            code_table,
            # top_k=request.top_k,
            # max_dist=request.max_dist
            )

        return RAGResponse(text = result)
    except Exception as e:
        logger.error(f"Unexpected error in getting RAG Results: {e}")
        raise HTTPException(status_code=500, detail=f"Error in getting RAG Results: {e}")


# Optional: Add a health check endpoint
@app.get("/health")
async def health_check():
    return {"status": "healthy"}

# Run App


Copy the `Public URL` in your local of `/config`. Everytime you run the below code, it'll be a different URL so need to change in the config file too

In [16]:
import torch, nest_asyncio, uvicorn
from pyngrok import ngrok

print("cuda" if torch.cuda.is_available() else "cpu")

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

cuda
Public URL: https://60a4-34-127-13-20.ngrok-free.app


INFO:     Started server process [383]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2401:4900:1c6f:1f53:2d30:6fe4:8273:c4b2:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2401:4900:1c6f:1f53:2d30:6fe4:8273:c4b2:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2401:4900:1c6f:1f53:2d30:6fe4:8273:c4b2:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2401:4900:1c6f:1f53:2d30:6fe4:8273:c4b2:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2401:4900:1c6f:1f53:2d30:6fe4:8273:c4b2:0 - "POST /python-leetcode-RAG HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [383]
